## Import Required Libraries

In [2]:
from nba_api.live.nba.endpoints import boxscore
import duckdb
import json
import pandas as pd
# Fetch the boxscore data
json_obj = json.loads(boxscore.BoxScore(game_id='0012400029').get_json())

# Extract game ID and team stats
game_id = json_obj['game']['gameId']
home_team_stats = json_obj['game']['homeTeam']
away_team_stats = json_obj['game']['awayTeam']

# Create a dictionary with the required information
game_data = {
    'game_id': game_id,
    'home_team_id': home_team_stats['teamId'],
    'home_team_name': f"{home_team_stats['teamCity']} {home_team_stats['teamName']}",
    'home_team_score': home_team_stats['score'],
    'away_team_id': away_team_stats['teamId'],
    'away_team_name': f"{away_team_stats['teamCity']} {away_team_stats['teamName']}",
    'away_team_score': away_team_stats['score']
}

# Add statistics for both teams
for stat, value in home_team_stats['statistics'].items():
    game_data[f'home_{stat}'] = value
for stat, value in away_team_stats['statistics'].items():
    game_data[f'away_{stat}'] = value

# Create the DataFrame
df = pd.DataFrame([game_data])
df.iloc[:,50:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 79 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   home_reboundsTotal                 1 non-null      int64  
 1   home_secondChancePointsAttempted   1 non-null      int64  
 2   home_secondChancePointsMade        1 non-null      int64  
 3   home_secondChancePointsPercentage  1 non-null      float64
 4   home_steals                        1 non-null      int64  
 5   home_threePointersAttempted        1 non-null      int64  
 6   home_threePointersMade             1 non-null      int64  
 7   home_threePointersPercentage       1 non-null      float64
 8   home_timeLeading                   1 non-null      object 
 9   home_timesTied                     1 non-null      int64  
 10  home_trueShootingAttempts          1 non-null      float64
 11  home_trueShootingPercentage        1 non-null      float64
 12

In [3]:
pd.DataFrame.from_dict(json_obj['game']['homeTeam']['statistics'], orient='index').transpose()

,assists,assistsTurnoverRatio,benchPoints,biggestLead,biggestLeadScore,biggestScoringRun,biggestScoringRunScore,blocks,blocksReceived,fastBreakPointsAttempted,...,timeLeading,timesTied,trueShootingAttempts,trueShootingPercentage,turnovers,turnoversTeam,turnoversTotal,twoPointersAttempted,twoPointersMade,twoPointersPercentage
0,23,2.3,20,28,89-117,10,89-117,4,5,9,...,PT27M27.00S,10,91.6,0.638646,9,1,10,46,29,0.630435


In [4]:
duckdb.sql("""
           WITH unnested_arrays AS (
           SELECT UNNEST(home_players) AS home_players, UNNEST(away_players) AS away_players
           FROM read_json("Game Statistics/0012400070_2024-10-19.jsonl", auto_detect = True))
           
           SELECT 
               home_players.name AS home_players, home_players.oncourt,
               away_players.name AS away_players, away_players.oncourt
           FROM unnested_arrays
           WHERE home_players.oncourt = 1 AND away_players.oncourt = 1
           """).to_df()

,home_players,oncourt,away_players,oncourt_2
0,Jevon Carter,1,Pete Nance,1
1,Chris Duarte,1,Craig Porter Jr.,1
2,Talen Horton-Tucker,1,Jules Bernard,1
3,Adama Sanogo,1,JT Thor,1
4,E.J. Liddell,1,Luke Travers,1


In [5]:
duckdb.sql("""
           WITH unnested_outer_arrays AS (
           SELECT UNNEST(home_players) AS home_players, UNNEST(away_players) AS away_players
           FROM read_json("Game Statistics/0012400070_2024-10-19.jsonl", auto_detect = True)),
           
           inner_arrays AS (
           SELECT 
               home_players.name AS player1,
               away_players.name AS player2,
               home_players.statistics AS home_player_stats,
               away_players.statistics AS away_players_stats
           FROM unnested_outer_arrays),
           
           home_player_stats AS(
           SELECT player1, home_player_stats.* EXCLUDE('twoPointersAttempted', 'twoPointersMade', 'twoPointersPercentage', 'minus', 'plus')
           FROM inner_arrays),
           
           away_player_stats AS(
           SELECT player2, home_player_stats.* EXCLUDE('twoPointersAttempted', 'twoPointersMade', 'twoPointersPercentage', 'minus', 'plus')
           FROM inner_arrays)
           
           SELECT 
                'home' AS team, player1 AS player, points AS points, assists AS assists, reboundsTotal || ' (Off: ' || reboundsOffensive || ' Def: ' || reboundsDefensive || ')' AS rebounds, blocks, steals, turnovers, fieldGoalsAttempted AS fg_attempts, fieldGoalsMade AS fg_made, ROUND(fieldGoalsPercentage, 3) AS fg_percentage, freeThrowsAttempted AS ft_attempts, freeThrowsMade AS ft_made, ROUND(freeThrowsPercentage, 3) AS ft_percentage, threePointersAttempted AS "3pt_attempts", threePointersMade AS "3pt_made", threePointersPercentage AS "3pt_percentage", pointsInThePaint AS paint_points, pointsSecondChance AS second_chance_points
           FROM home_player_stats
           WHERE player IS NOT NULL
           UNION 
           SELECT 
                'away' AS team, player2 AS player, points AS points, assists AS assists, reboundsTotal || ' (Off: ' || reboundsOffensive || ' Def: ' || reboundsDefensive || ')' AS rebounds, blocks, steals, turnovers, fieldGoalsAttempted AS fg_attempts, fieldGoalsMade AS fg_made, ROUND(fieldGoalsPercentage, 3) AS fg_percentage, freeThrowsAttempted AS ft_attempts, freeThrowsMade AS ft_made, ROUND(freeThrowsPercentage, 3) AS ft_percentage, threePointersAttempted AS "3pt_attempts", threePointersMade AS "3pt_made", threePointersPercentage AS "3pt_percentage", pointsInThePaint AS paint_points, pointsSecondChance AS second_chance_points
           FROM away_player_stats
           WHERE player IS NOT NULL AND points IS NOT NULL
           ORDER BY team, points DESC, assists DESC, rebounds DESC
           """).to_df()

,team,player,points,assists,rebounds,blocks,steals,turnovers,fg_attempts,fg_made,fg_percentage,ft_attempts,ft_made,ft_percentage,3pt_attempts,3pt_made,3pt_percentage,paint_points,second_chance_points
0,away,Darius Garland,19,3,3 (Off: 1 Def: 2),0,1,0,11,7,0.636,2,0,0.000,7,5,0.714286,4,0
1,away,Jarrett Allen,18,6,12 (Off: 2 Def: 10),1,1,3,16,8,0.500,1,1,1.000,3,1,0.333333,12,2
2,away,Dean Wade,18,3,6 (Off: 1 Def: 5),0,1,1,14,7,0.500,4,2,0.500,7,2,0.285714,10,0
3,away,Donovan Mitchell,14,8,6 (Off: 1 Def: 5),0,2,5,14,5,0.357,2,2,1.000,8,2,0.250000,4,0
4,away,Georges Niang,14,4,9 (Off: 3 Def: 6),0,1,1,9,5,0.556,6,4,0.667,2,0,0.000000,10,4
5,away,Caris LeVert,11,3,3 (Off: 0 Def: 3),0,0,1,9,4,0.444,0,0,0.000,7,3,0.428571,0,0
6,away,Evan Mobley,10,3,6 (Off: 0 Def: 6),1,0,0,8,4,0.500,0,0,0.000,5,2,0.400000,2,0
7,away,Pete Nance,9,1,1 (Off: 0 Def: 1),0,0,1,5,3,0.600,2,2,1.000,2,1,0.500000,4,2
8,away,Jules Bernard,9,0,2 (Off: 0 Def: 2),0,0,1,8,3,0.375,2,2,1.000,3,1,0.333333,2,0
9,away,Ty Jerome,7,0,3 (Off: 0 Def: 3),0,0,1,4,3,0.750,0,0,0.000,1,1,1.000000,4,0


In [11]:
duckdb.sql("""
           SELECT home_team.* EXCLUDE(biggestLeadScore, biggestScoringRun, biggestScoringRunScore, foulsPersonal, foulsTechnical, leadChanges, minutes, minutesCalculated, pointsAgainst, reboundsPersonal, timeLeading, timesTied, team_id, team_name, blocksReceived, twoPointersAttempted, twoPointersMade, twoPointersPercentage, fastBreakPointsAttempted, fastBreakPointsMade, fastBreakPointsPercentage, turnoversTeam, turnovers, pointsInThePaintAttempted, pointsInThePaintMade, pointsInThePaintPercentage, secondChancePointsAttempted, secondChancePointsMade, secondChancePointsPercentage)
           FROM read_json('Game Statistics/0012400067_2024-10-19.jsonl', auto_detect=True)
           """).to_df().columns

Index(['assists', 'assistsTurnoverRatio', 'benchPoints', 'biggestLead',
       'blocks', 'fieldGoalsAttempted', 'fieldGoalsEffectiveAdjusted',
       'fieldGoalsMade', 'fieldGoalsPercentage', 'foulsOffensive',
       'foulsDrawn', 'foulsTeam', 'foulsTeamTechnical', 'freeThrowsAttempted',
       'freeThrowsMade', 'freeThrowsPercentage', 'points', 'pointsFastBreak',
       'pointsFromTurnovers', 'pointsInThePaint', 'pointsSecondChance',
       'reboundsDefensive', 'reboundsOffensive', 'reboundsTeam',
       'reboundsTeamDefensive', 'reboundsTeamOffensive', 'reboundsTotal',
       'steals', 'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'trueShootingAttempts',
       'trueShootingPercentage', 'turnoversTotal'],
      dtype='object')

In [29]:

import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import duckdb

file = 'Game Statistics/0012400071_2024-10-19.jsonl'

# Sample data
categories = duckdb.sql(f"""
                        WITH stat_categories AS(
           SELECT home_team.* EXCLUDE(biggestLeadScore, biggestScoringRun, biggestScoringRunScore, foulsPersonal, foulsTechnical, leadChanges, minutes, minutesCalculated, pointsAgainst, reboundsPersonal, timeLeading, timesTied, team_id, team_name, blocksReceived, twoPointersAttempted, twoPointersMade, twoPointersPercentage, fastBreakPointsAttempted, fastBreakPointsMade, fastBreakPointsPercentage, turnoversTeam, turnovers, pointsInThePaintAttempted, pointsInThePaintMade, pointsInThePaintPercentage, secondChancePointsAttempted, secondChancePointsMade, secondChancePointsPercentage)
           FROM read_json('{file}', auto_detect=True))
           
           SELECT 
                reboundsOffensive + reboundsTeamOffensive AS 'Offensive Rebounds',
                reboundsDefensive + reboundsTeamDefensive AS 'Defensive Rebounds',
                ROUND(ROUND(trueShootingPercentage, 4)*100, 2) AS 'True Shooting %',
                ROUND(ROUND(fieldGoalsEffectiveAdjusted, 4)*100, 2) AS 'Effective FG %',
                ROUND(assistsTurnoverRatio, 3) AS 'Assist-Turnover Ratio',
                pointsFromTurnovers AS 'Points From Turnovers',
                pointsFastBreak AS 'Fast Break Pts',
                pointsSecondChance AS 'Second Chance Pts',
                pointsInThePaint AS 'Paint Points',
                ROUND(ROUND(threePointersPercentage, 4)*100,2) AS '3PT %',
                ROUND(freeThrowsPercentage, 4)*100 AS 'Free Throw %',
                ROUND(fieldGoalsPercentage, 4)*100 AS 'Field Goal %',
                turnoversTotal AS 'Turnovers',
                steals AS 'Steals',
                blocks AS 'Blocks',
                reboundsTotal AS 'Rebounds',
                assists AS 'Assists',
                points AS 'Points',           
           FROM stat_categories
           """).to_df().columns.tolist()
inverse_categories = ['Turnovers']
home_team_name = duckdb.sql(f"""
                            SELECT home_team.team_name
                            FROM read_json('{file}', auto_detect=True)
                            """).to_df().iloc[0].iloc[0]
away_team_name = duckdb.sql(f"""
                            SELECT away_team.team_name
                            FROM read_json('{file}', auto_detect=True)
                            """).to_df().iloc[0].iloc[0]
home_stats = duckdb.sql(f"""
            WITH stat_categories AS(
                SELECT home_team.* EXCLUDE(biggestLeadScore, biggestScoringRun, biggestScoringRunScore,     foulsPersonal, foulsTechnical, leadChanges, minutes, minutesCalculated, pointsAgainst, reboundsPersonal, timeLeading, timesTied, team_id, team_name, blocksReceived, twoPointersAttempted, twoPointersMade, twoPointersPercentage, fastBreakPointsAttempted, fastBreakPointsMade, fastBreakPointsPercentage, turnoversTeam, turnovers, pointsInThePaintAttempted, pointsInThePaintMade, pointsInThePaintPercentage, secondChancePointsAttempted, secondChancePointsMade, secondChancePointsPercentage)
           FROM read_json('{file}', auto_detect=True))
           
            SELECT 
                reboundsOffensive + reboundsTeamOffensive AS 'Offensive Rebounds',
                reboundsDefensive + reboundsTeamDefensive AS 'Defensive Rebounds',
                ROUND(ROUND(trueShootingPercentage, 4)*100, 2) AS 'True Shooting %',
                ROUND(ROUND(fieldGoalsEffectiveAdjusted, 4)*100, 2) AS 'Effective FG %',
                ROUND(assistsTurnoverRatio, 3) AS 'Assist-Turnover Ratio',
                pointsFromTurnovers AS 'Points From Turnovers',
                pointsFastBreak AS 'Fast Break Pts',
                pointsSecondChance AS 'Second Chance Pts',
                pointsInThePaint AS 'Paint Points',
                ROUND(ROUND(threePointersPercentage, 4)*100, 2) AS '3PT %',
                ROUND(freeThrowsPercentage, 4)*100 AS 'Free Throw %',
                ROUND(fieldGoalsPercentage, 4)*100 AS 'Field Goal %',
                turnoversTotal AS 'Turnovers',
                steals AS 'Steals',
                blocks AS 'Blocks',
                reboundsTotal AS 'Rebounds',
                assists AS 'Assists',
                points AS 'Points',           
           FROM stat_categories
           """).to_df().values.tolist()[0]
away_stats = duckdb.sql(f"""                        WITH stat_categories AS(
           SELECT away_team.* EXCLUDE(biggestLeadScore, biggestScoringRun, biggestScoringRunScore, foulsPersonal, foulsTechnical, leadChanges, minutes, minutesCalculated, pointsAgainst, reboundsPersonal, timeLeading, timesTied, team_id, team_name, blocksReceived, twoPointersAttempted, twoPointersMade, twoPointersPercentage, fastBreakPointsAttempted, fastBreakPointsMade, fastBreakPointsPercentage, turnoversTeam, turnovers, pointsInThePaintAttempted, pointsInThePaintMade, pointsInThePaintPercentage, secondChancePointsAttempted, secondChancePointsMade, secondChancePointsPercentage)
           FROM read_json('{file}', auto_detect=True))
           
           SELECT 
                reboundsOffensive + reboundsTeamOffensive AS 'Offensive Rebounds',
                reboundsDefensive + reboundsTeamDefensive AS 'Defensive Rebounds',
                ROUND(ROUND(trueShootingPercentage, 4)*100, 2) AS 'True Shooting %',
                ROUND(ROUND(fieldGoalsEffectiveAdjusted, 4)*100, 2) AS 'Effective FG %',
                ROUND(assistsTurnoverRatio, 3) AS 'Assist-Turnover Ratio',
                pointsFromTurnovers AS 'Points From Turnovers',
                pointsFastBreak AS 'Fast Break Pts',
                pointsSecondChance AS 'Second Chance Pts',
                pointsInThePaint AS 'Paint Points',
                ROUND(ROUND(threePointersPercentage, 4)*100, 2) AS '3PT %',
                ROUND(freeThrowsPercentage, 4)*100 AS 'Free Throw %',
                ROUND(fieldGoalsPercentage, 4)*100 AS 'Field Goal %',
                turnoversTotal AS 'Turnovers',
                steals AS 'Steals',
                blocks AS 'Blocks',
                reboundsTotal AS 'Rebounds',
                assists AS 'Assists',
                points AS 'Points',           
           FROM stat_categories
           """).to_df().values.tolist()[0]

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(f'{home_team_name} vs {away_team_name} - Game Statistics', style={'color': 'white'}),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': cat, 'value': cat} for cat in categories] + [{'label': 'Show All', 'value': 'all'}],
        value=['all'],
        multi=True
    ),
    dcc.Graph(id='stats-graph')
])

@app.callback(
    Output('stats-graph', 'figure'),
    Input('category-dropdown', 'value')
)
def update_graph(selected_categories):
    if 'all' in selected_categories:
        selected_categories = categories
    
    fig = go.Figure()

    home_colors = [
        'green' if (a >= b and category not in inverse_categories) or (a < b and category in inverse_categories) 
        else 'red' 
        for a, b, category in zip(home_stats, away_stats, categories)
        if category in selected_categories
    ]
    away_colors = [
        'green' if (a <= b and category not in inverse_categories) or (a > b and category in inverse_categories) 
        else 'red' 
        for a, b, category in zip(home_stats, away_stats, categories)
        if category in selected_categories
    ]

    fig.add_trace(go.Bar(
        y=[cat for cat in categories if cat in selected_categories], 
        x=[-val for val, cat in zip(home_stats, categories) if cat in selected_categories],
        name=home_team_name,
        orientation='h',
        marker_color=home_colors,
        hovertemplate=[f"{home_team_name}<br>{cat}: {val}" for cat, val in zip(categories, home_stats) if cat in selected_categories],
    ))

    fig.add_trace(go.Bar(
        y=[cat for cat in categories if cat in selected_categories],
        x=[val for val, cat in zip(away_stats, categories) if cat in selected_categories],
        name=away_team_name,
        orientation='h',
        marker_color=away_colors,
        hovertemplate=[f'{away_team_name}<br>{cat}: {val}<extra></extra>' for cat, val in zip(categories, away_stats) if cat in selected_categories],
    ))

    fig.update_layout(
        barmode='overlay',
        xaxis=dict(
            showgrid=False,
            showticklabels=False
        ),
        yaxis=dict(showgrid=False),
        bargap=0.2,
        bargroupgap=0.1,
        plot_bgcolor='white',
        showlegend=True
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
